# Tool Integration for Agents

In [2]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os

print("All imports successful")

All imports successful


In [3]:
load_dotenv()
api_key = os.getenv("paid_api")

if not api_key:
    raise ValueError("API Key not found!")
print("API Key loaded successfully")

API Key loaded successfully


In [4]:
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0.7,
    api_key=api_key
)

print(f"LLM initialized {llm.model_name}")

LLM initialized gpt-4o-mini


## First Tool

The @tool decorator converts a python function into a tool that LLM can call

In [5]:
@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression and return the result.
    Use this tool when you need to perform calculations.
    
    Args:
        expression: A mathematical expression like "2 + 2" or "15 * 37"
        
    Returns:
        The calculated result as a string
        
    Examples:
        - "2 + 2" returns "4"
        - "100 / 5" returns "20.0"
        - "2 ** 10" returns "1024"
    """

    try: 
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error calculting: {str(e)}"
    
print("Calculator tool created")

Calculator tool created


In [6]:
# Test the calculator tool
result = calculator.invoke({"expression": "123 * 456"})
print(f"123 * 456 = {result}")

result2 = calculator.invoke("2**10")
print(f"2^10 = {result2}")

123 * 456 = 56088
2^10 = 1024


## Creating a Second Tool

In [7]:
@tool
def text_analyzer(text: str) -> str:
    """
    Analyze text and return statistics about it.
    Use this tool when you need to analyze or count things in text.
    
    Args:
        text: The text to analyze
        
    Returns:
        Statistics about the text (characters, words, sentences)
        
    Examples:
        - "Hello world" returns character count, word count, etc.
    """
    char_count = len(text)
    word_count = len(text.split())
    sentence_count = text.count('.') + text.count('!') + text.count('?')
    return f"""Text analysis:
- Characters: {char_count}
- Words: {word_count}
- Sentences: {sentence_count}
- First 50 chars: {text[:50]}..."""

print(f"Text analyzer tool created")

Text analyzer tool created


In [8]:
test_text = "Hello! This is a test. How are you today?"
result = text_analyzer.invoke({"text": test_text})
print(result)

Text analysis:
- Characters: 41
- Words: 9
- Sentences: 3
- First 50 chars: Hello! This is a test. How are you today?...


## Binding Tools to the LLM

In [9]:
tools = [calculator, text_analyzer]

llm_with_tools = llm.bind_tools(tools)
print(f"LLM bound to {len(tools)} tools")
print(f"Tools: {[tool.name for tool in tools]}")

LLM bound to 2 tools
Tools: ['calculator', 'text_analyzer']


### Test: LLM decisiono making

In [10]:
# Test: Does the LLM decide to call calculator?
response = llm_with_tools.invoke([HumanMessage(content="What is 234*456?")])

print(f"Response type: {type(response)}")
print(f"\nContent: {response.content}")
print(f"\nTool calls: {response.tool_calls}")

Response type: <class 'langchain_core.messages.ai.AIMessage'>

Content: 

Tool calls: [{'name': 'calculator', 'args': {'expression': '234 * 456'}, 'id': 'call_np8gWOzYDQRPYSMpfaSWZbzT', 'type': 'tool_call'}]


In [11]:
# Test: Does LLM decide not to call tools for simple queries?
response2 = llm_with_tools.invoke([HumanMessage(content="Hello! How are you?")])
print(f"Content: {response2.content}")
print(f"Tool calls: {response2.tool_calls}")

Content: Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?
Tool calls: []


## Building the Agent Graph

### Define the Assitant Node

In [ ]:
sys_msg = SystemMessage(content="""You are a helpful assitant with access to tools.
When asked to perform calculations, use the calculator tool.
When asked to analyze text, use the text_analyzer tool

Only use tools when necessary - for simple questions, answer directly.""")

def assistant(state: MessagesState) -> dict:
    """
    Assistant node - decides whether to use tools or answer directly.
    """
    messages = [sys_msg] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

print("Assistant mode defined")

### Define Conditional Routing

In [ ]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Decide next step based on last message.
    
    If LLM called a tool → go to 'tools' node
    If LLM provided final answer → go to END
    """
    last_message = state["messages"][-1]

    #Check if LLM made tool calls

SyntaxError: expected ':' (1806659691.py, line 1)